# Lecture 30: Threshold Accepting

```{note}
...
```
---

## Overview

Much like the Tabu Search algorithm, the Threshold Accepting algorithm is designed to search for high-quality solutions in complex search space while avoiding local optimal traps. However, unlike the Tabu Search, this algorithm does so with fewer degrees of freedom (input parameters). It starts by initialising the current solution – $s$, and the best solution – $s^*$, from the given initial solution – $s_o$. The algorithm then iterates searching the solution space for better solutions until it has achieved threshold level of solution quality or computation effort. In each such iteration, the algorithm generates a random new solution by applying a defined set of transformations to the current solution defined by the neighbourhood – $N(s)$. Notably, unlike any of the previously discussed algorithms, the Threshold Accepting algorithm accepts this new solution as the current solution even if it is somewhat worse than the current solution. Specifically, if the difference in objective function values of the new and the current solution is less than a predefined threshold – $Δ (Δ>0)$, then the algorithm sets the current solution to this new solution. However, the algorithm updates the best solution only if the new solution is better than the best. Finally, upon convergence, the Threshold Accepting algorithm returns the best solution.

The Threshold Accepting algorithm balances solution diversification (exploration) by accepting certain inferior solutions as the current solution, and solution intensification (exploitation) by only accepting better solutions as the best solution. And while the Tabu Search achieves this balance by maintaining a dedicated list of “tabooed” and “accepted” solutions, the Threshold Accepting algorithm does so with just one parameter – $Δ$. This allows for fast implementation with low computational effort due to minimal fine-tuning and memory requirements. Nonetheless, if the threshold is set too high, then the algorithm will have a slow convergence (exploration bias). On the other hand, it the threshold is set too low, then the algorithm will get stuck on a local optimal (exploitation bias). Thus, with appropriately sized threshold, the algorithm can swiftly converge to high quality solutions, even for problems with complex multimodal solution landscape.

---

## Pseudo Code

1. **Procedure** $\text{TA}(s_o, (N, \Delta))$
2. $s ← s_o$ &emsp;<small>// initialize current solution $s$ as the initial solution $s_o$</small>
3. $s^* ← s$ &emsp;<small>// initialize best solution $s^*$ as the current solution</small>
4. **while** $!converged$ **do** &emsp;<small>// repeat until converged</small>
5. &emsp;$s' {R \atop ←} N(s)$ &emsp;<small>// generate new random solution from neighborhood $N$ of the current solution</small>
6. &emsp;$\delta ← f(s') - f(s)$ &emsp;<small>// calculate the difference in objective values</small>
7. &emsp;**if** $\delta < \Delta$ **then** &emsp;<small>// if the new solution is within threshold $\Delta$ of the current solution</small>
8. &emsp;&emsp;$s ← s'$ &emsp;<small>// update the current solution to the new solution</small>
9. &emsp;**end if**
10. &emsp;**if** $f(s) < f(s^*)$ **then** &emsp;<small>// if the current solution is better than the best solution</small>
11. &emsp;&emsp;$s^* ← s$ &emsp;<small>// update the best solution to the current solution</small>
12. &emsp;**end if**
13. **end while**
14. **return** $s^*$ &emsp;<small>// return the best solution</small>

---

## Implementation

---

## Case Study

---

```{note}
...
```